## KERAS - Pre Trained Models

In [1]:
# from keras.applications.xception import Xception
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.resnet import ResNet50
# from keras.applications.resnet import ResNet101
# from keras.applications.resnet import ResNet152
# from keras.applications.resnet_v2 import ResNet50V2
# from keras.applications.resnet_v2 import ResNet101V2
# from keras.applications.resnet_v2 import ResNet152V2
# from keras.applications.resnext import ResNeXt50
# from keras.applications.resnext import ResNeXt101
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications.mobilenet import MobileNet
# from keras.applications.mobilenet_v2 import MobileNetV2
# from keras.applications.densenet import DenseNet121
# from keras.applications.densenet import DenseNet169
# from keras.applications.densenet import DenseNet201
# from keras.applications.nasnet import NASNetLarge
# from keras.applications.nasnet import NASNetMobile

### Importing Libraries

In [1]:
from pathlib import Path
import numpy as np
from keras.preprocessing import image
import requests
import sys, os
import json
import pprint
import pandas as pd
from PIL import Image
import json
import cv2
import matplotlib.pyplot as plt
import image_preprocessing_library as lib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# appending common pakcage in the sys.path for module to be found during imports
# add this line to other files as well, IMP! it is. 
#sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'..','common'))
#from common import helpers

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [2]:
ptms = {}

from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input as resnet50_preprocess_input
from keras.applications.resnet import decode_predictions as resnet50_decode_predictions
def get_resnet50():
    return (ResNet50(weights='imagenet'), resnet50_preprocess_input, resnet50_decode_predictions)
ptms["resnet50"] = get_resnet50


from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.applications.vgg19 import decode_predictions as vgg19_decode_predictions
def get_vgg19():
    return (VGG19(weights='imagenet'), vgg19_preprocess_input, vgg19_decode_predictions)
ptms["vgg19"] = get_vgg19


from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_preprocess_input
from keras.applications.xception import decode_predictions as xception_decode_predictions
def get_xception():
    return (Xception(weights='imagenet'), xception_preprocess_input, xception_decode_predictions)
ptms["xception"] = get_xception

from keras.applications.resnet import ResNet101
from keras.applications.resnet import preprocess_input as resnet101_preprocess_input
from keras.applications.resnet import decode_predictions as resnet101_decode_predictions
def get_resnet101():
    return (ResNet101(weights='imagenet'), resnet101_preprocess_input, resnet101_decode_predictions)
ptms["resnet101"] = get_resnet101

from keras.applications.resnet import ResNet152
from keras.applications.resnet import preprocess_input as resnet152_preprocess_input
from keras.applications.resnet import decode_predictions as resnet152_decode_predictions
def get_resnet152():
    return (ResNet152(weights='imagenet'), resnet152_preprocess_input, resnet152_decode_predictions)
ptms["resnet152"] = get_resnet152

from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.resnet_v2 import preprocess_input as resnet50v2_preprocess_input
from keras.applications.resnet_v2 import decode_predictions as resnet50v2_decode_predictions
def get_resnet50v2():
    return (ResNet50V2(weights='imagenet'), resnet50v2_preprocess_input, resnet50v2_decode_predictions)
ptms["resnet50v2"] = get_resnet50v2

from keras.applications.resnet_v2 import ResNet101V2
from keras.applications.resnet_v2 import preprocess_input as resnet101v2_preprocess_input
from keras.applications.resnet_v2 import decode_predictions as resnet101v2_decode_predictions
def get_resnet101v2():
    return (ResNet101V2(weights='imagenet'), resnet101v2_preprocess_input, resnet101v2_decode_predictions)
ptms["resnet101v2"] = get_resnet101v2

### Pre Processing Sequence

In [3]:
pre_processing_seq_dict = {
    "seq_0" : [], # for raw seq
    "seq_1" : ["gray"],
    "seq_2" : ["hsv"],
    "seq_3" : ["sharpen"],
    "seq_4" : ["gray", "bilateral_blur", "threshold_mean"],
    "seq_5" : ["gray", "bilateral_blur", "threshold_gaussian"],
    "seq_6" : ["gray", "bilateral_blur", "threshold_otsu"],
    "seq_7" : ["median_blur"],
    "seq_8" : ["gaussian_blur"],
    "seq_9" : ["bilateral_blur"],
    "seq_10" : ["fastnl_blur"],
    "seq_11" : ["gray", "bilateral_blur", "threshold_otsu", "opening"],
    "seq_12" : ["gray", "bilateral_blur", "threshold_otsu", "closing"],
    "seq_13" : ["opening"],
    "seq_14" : ["closing"],
    "seq_15" : ["gray", "sobel"],
    "seq_16" : ["gray", "laplacian"],
    "seq_17" : ["gray", "canny"]
}

### Config/Map Files Load
* change below paths while running on GPU
* you may prefer to dump config files/jsons in the same notebook dir

In [4]:
# dictionary containing imagenet - custom label maps
imagenet_label_map = None
with open('./config_jsons/imagenet_label_map.json') as json_file: 
    imagenet_label_map = json.load(json_file)

# dictionary containing model ids - model_name map
model_ids = None
with open('./config_jsons/model_ids_map.json') as json_file: 
    model_ids = json.load(json_file)

# dictionary containing dataset_ids and dataset_desc    
dataset_ids = None
with open('../../dataset/image_classification/dataset_id_map.json') as json_file:
    dataset_ids = json.load(json_file)
    
custom_dataset_labels = None
with open('./config_jsons/label_ids_map.json') as json_file:
    custom_dataset_labels = json.load(json_file)

#### Consider making below cell parameterised later using papermill, or you may prefer to do it manually

In [5]:
# taking path as a parameter here, to ease out refactoring in future.
# first experimentation is on raw dataset, will scale out to others
dataset_path = Path("../../dataset/image_classification/raw")
master_df_keras_path = "./experiment_results/master_keras.csv"

In [6]:
# dataframe columns
columns = ["model_id", "model_name", "seq_id", "seq_name", "image_name", "label_id", 
             "pred_label_id", "label_name","pred_label_name", "pred_confidence"]

In [8]:
# df = pd.DataFrame(columns = columns)
# df.to_csv(master_df_keras_path, index=False)

In [9]:
custom_labels = [x for x in os.walk(dataset_path)][0][1]

In [10]:
models_trained_on_299x299 = [
    "inception_v3", 
    "xception"
]

### Functions Definitions

#### OpenCV to PIL image conversion

In [11]:
def convert_to_pil_img(opencv_img):
    if opencv_img.dtype == 'float64':
        opencv_img = opencv_img.astype(np.uint8)
    pil_img = cv2.cvtColor(opencv_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(pil_img)
    return pil_img

def convert_to_cv_img(pil_img):
    np_img_arr = np.asarray(pil_img)
    cv_image=cv2.cvtColor(np_img_arr, cv2.COLOR_RGB2BGR)
    return cv_image

In [12]:
def get_model_id(model_name):
    for id, name in model_ids.items():
        if name == model_name:
            return id
    return "not found for model name: " + model_name

def get_model_name(id):
    if id not in model_ids.keys():
        return "not found for model id: " + str(id)
    return model_ids[id]

def get_seq_operations(seq_id):
    return pre_processing_seq_dict[seq_id]

def get_seq_name(seq_id):
    if seq_id not in pre_processing_seq_dict.keys():
        return "not found for dataset id: " + seq_id
    return " > ".join(get_seq_operations(seq_id))

def get_label_id(label_name):
    for id, name in custom_dataset_labels.items():
        if name == label_name:
            return id
    return "not found for label: "+label_name

def get_label_name(id):
    if id not in custom_dataset_labels.keys():
        return "not found for id: " + str(id)
    return custom_dataset_labels[id]

def get_pred_label_id(pred_label_name):
    for custom_label, imagenet_labels in imagenet_label_map.items():
        if pred_label_name.lower() in imagenet_labels:
            return get_label_id(custom_label)
    return get_label_id("others")

def load_master_df_from_csv():
    return pd.read_csv(master_df_keras_path)

def dump_master_df_to_csv(master_df):
    master_df.to_csv(master_df_keras_path, header=True, columns=columns, index=False)       

def log_per_label_results_to_csv(model_name, label_name, pred_result_list, seq_id):
    # input - pred_result_list: (label, pred_tuple_list), pred_tuple_list: [(image_name, pred_label, pred_percentage)]
    # columns = ["model_id", "model_name", "dataset_id", "dataset_name", "image_name", "label_id", 
    #            "pred_label_id", "label_name","pred_label_name", "pred_confidence"]

    model_id = get_model_id(model_name)
    model_name = model_name
    # change below before running on pre processed image dataset
    seq_id = seq_id
    seq_name = get_seq_name(seq_id)
    label_id = get_label_id(label_name)
    label_name = label_name
    
    rows = []
    for res in pred_result_list:
        image_name = res[0]
        pred_label_name = res[1]
        pred_confidence = res[2]
        pred_label_id = get_pred_label_id(pred_label_name)
        row = [model_id, model_name, seq_id, seq_name, image_name, label_id, label_name, 
               pred_label_id, pred_label_name, pred_confidence]
        rows.append(row)
    
    current_label_df = pd.DataFrame(rows, columns = columns)
    master_df = load_master_df_from_csv()
    master_df = master_df.append(current_label_df)
    dump_master_df_to_csv(master_df)

In [13]:
def load_per_label_imgs_generator(size_default, seq_id):
    # better due to memory restrictions we read per class images at once i.e airplane folder at a time.
    # output: (label, [(image_name, pil_image)])
    for label in custom_labels:
        per_label_images = []
        label_image_names = [x for x in os.walk(dataset_path/label)][0][2]
        for image_name in label_image_names:
            img_path = dataset_path/label/image_name
            if size_default:
                img = image.load_img(img_path, color_mode="rgb", target_size=(224, 224))
            else:
                img = image.load_img(img_path, color_mode="rgb", target_size=(299, 299))
            img = apply_cv_transformations(seq_id, img)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            per_label_images.append((image_name, x))
        yield (label, per_label_images)

def apply_cv_transformations(seq_id, pil_img):
    cv_img = convert_to_cv_img(pil_img)
    operations = get_seq_operations(seq_id)
    processed_img = cv_img
    for operation in operations:
        processed_img = lib.dispatcher[operation](processed_img)
    return convert_to_pil_img(processed_img) 
    
        
def transform_images(preprocess_method, imgs_tuple):
    # input: imgs_tuple: (image_name, pil_image)
    # output: processed_imgs: (image_name, processed_image)
    processed_imgs = []
    
    for img_tuple in imgs_tuple:
        img = preprocess_method(img_tuple[1])
        processed_imgs.append((img_tuple[0],img))
    return processed_imgs


def evaluate_results(model, imgs, decode_predictions):
    # input: imgs = (img_name, pil_image)
    # output: results = [(img_name, pred_label, prob)]
    results = []
    for img in imgs:
        pred = model.predict(img[1])
        (id, label, label_prob) = decode_predictions(pred, top=1)[0][0]
        results.append((img[0], label, label_prob))
    return results


def run_per_label_model_inference(model, model_name, preprocess_method, decode_predictions, default_size, seq_id):
    # input - model: which model?, preprocess_method: model dependent method, decode_predictions: model dependent
    # input - default_size = true, means load image in 299x299 or 224x224?
    # imgs_tuple: (label, [images])
    # output - (label, pred_tuple_list), here pred_tuple_list: [(image_name, pred_label, prob)]
    for imgs_tuple in load_per_label_imgs_generator(default_size, seq_id):
        processed_imgs = transform_images(preprocess_method, imgs_tuple[1])
        pred_res = evaluate_results(model, processed_imgs, decode_predictions)
        label_name = imgs_tuple[0]
        log_per_label_results_to_csv(model_name, label_name, pred_res, seq_id)
    
def model_generator():
    for model_name in ptms.keys():
        yield (model_name, ptms[model_name])    

def run_all_model_inference():
    for model_tuple in model_generator():
        #model_tuple = next(model_generator())
        model_name = model_tuple[0]
        model = model_tuple[1]()
        default_size = True
        if model_name in models_trained_on_299x299:
            default_size = False
        print("Inference started for model: "+model_name)
        for seq_id in pre_processing_seq_dict.keys():
            print("inference started for seq_id: " + seq_id + ", ", end='')
            run_per_label_model_inference(model[0], model_name, model[1], model[2], default_size, seq_id)
            print("completed!")
        print("Inference completed! -----------------------------------")

### Model Evaluation

In [14]:
# need to change the save master_df path and assign model ids as well
run_all_model_inference()













Inference started for model: resnet50
inference started for seq_id: seq_0, completed!
inference started for seq_id: seq_1, completed!
Inference completed! -----------------------------------
Inference started for model: vgg19
inference started for seq_id: seq_0, completed!
inference started for seq_id: seq_1, completed!
Inference completed! -----------------------------------
Inference started for model: xception
inference started for seq_id: seq_0, 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-6433139424a4>", line 2, in <module>
    run_all_model_inference()
  File "<ipython-input-13-ff3e0af51c62>", line 76, in run_all_model_inference
    run_per_label_model_inference(model[0], model_name, model[1], model[2], default_size, seq_id)
  File "<ipython-input-13-ff3e0af51c62>", line 57, in run_per_label_model_inference
    pred_res = evaluate_results(model, processed_imgs, decode_predictions)
  File "<ipython-input-13-ff3e0af51c62>", line 44, in evaluate_results
    pred = model.predict(img[1])
  File "C:\Users\shubham\AppData\Roaming\Python\Python37\site-packages\keras\engine\training.py", line 1401, in predict
    callbacks=callbacks)
  File "C:\Users\shubham\AppData\Roaming\Python\Python37\site-packages\keras\engine\training_arrays.py", line 332, in predict_

KeyboardInterrupt: 